# Bamboo Report 0.1a

### Contents:
- Reference engine simulation
- Variable water content simulation (not done)
- Variable channel height simulation (not done)

In [1]:
#Import required modules
import bamboo as bam
import bamboo.cooling as cool
import bamboo.materials

import numpy as np
import matplotlib.pyplot as plt
import pypropep as ppp
import bamboo.plot
import thermo

# Reference engine
### *Reference engine setup*

In [4]:
'''Chamber conditions'''
pc = 15e5                   #Chamber pressure (Pa)
mdot = 5.4489               #Mass flow rate (kg/s)
p_amb = 1.01325e5           #Ambient pressure (Pa). 1.01325e5 is sea level atmospheric.
OF_ratio = 3.5              #Oxidiser/fuel mass ratio
water_mass_fraction = 0.10  #Fraction of the fuel that is water, by mass

'''Get combustion properties from pypropep'''
ppp.init()
e = ppp.Equilibrium()
ipa = ppp.PROPELLANTS['ISOPROPYL ALCOHOL']
water = ppp.PROPELLANTS['WATER']
n2o = ppp.PROPELLANTS['NITROUS OXIDE']

#Add propellants by mass fractions (note the mass fractions can add up to more than 1)
e.add_propellants_by_mass([(ipa, 1-water_mass_fraction), (water, water_mass_fraction), (n2o, OF_ratio)])

#Adiabatic combustion using chamber pressure                      
e.set_state(P = pc/1e5, type='HP')                      

gamma = e.properties.Isex   #pyrpopep uses 'Isex' for the ratio of specific heats
cp = 1000*e.properties.Cp   #Cp is given in kJ/kg/K, we want J/kg/K
Tc = e.properties.T

'''Set up the engine object'''
perfect_gas = bam.PerfectGas(gamma = gamma, cp = cp)    #Gas for frozen flow
chamber_conditions = bam.ChamberConditions(pc, Tc, mdot)
nozzle = bam.Nozzle.from_engine_components(perfect_gas, chamber_conditions, p_amb, type = "cone")
engine = bam.Engine(perfect_gas, chamber_conditions, nozzle)

print(f"\nNozzle throat area = {nozzle.At} m^2, nozzle exit area = {nozzle.Ae} m^2")

'''Check the thrust and specific impulse'''
print("")
print(f"Chamber Temperature: {Tc} K")
print(f"Gamma = {gamma}")
print(f"cp = {cp} J/kg/K")
print("")
print(f"Sea level thrust = {engine.thrust(1e5)/1000} kN      (using perfect gas relations)")
print(f"Sea level Isp = {engine.isp(1e5)} s         (using perfect gas relations)")



Loaded 1921 thermo species
Loaded 1031 propellants


ValueError: Nozzle type 'cone' is not currently implemented. Try 'conical' or 'rao'

### *Reference cooling simulation*

In [ ]:
'''Chamber and wall geometry'''
Ac = np.pi*0.1**2                      #Chamber cross-sectional area (m^2)
wall_thickness = 1e-3                  #Thickness of the metal engine liner (m)
L_star = 1.2                           #L_star = Volume_c/Area_t
chamber_length = L_star*nozzle.At/Ac   #Length of the combustion chamber
engine.add_geometry(chamber_length, Ac, wall_thickness)

'''Coolant jacket'''
mdot_coolant = mdot/(OF_ratio + 1)               #Coolant mass flow rate
inlet_T = 298.15                                 #Coolant inlet temperature
inlet_p0 = 200e5                                 #Tank / inlet coolant stagnation pressure (Pa)
wall_material = bam.materials.StainlessSteel304  #Stainless steel wall
#channel_width = 3e-2                             #Rectangular coolant channel width (m)
channel_height = 2e-3                            #Rectangular channel height (m)

#Choose the models we want to use for transport properties of the coolant and exhaust gas
thermo_coolant = thermo.mixture.Mixture(['isopropanol', 'water'], ws = [1 - water_mass_fraction, water_mass_fraction])
thermo_gas = thermo.mixture.Mixture(['N2', 'H2O', 'CO2'], zs = [e.composition['N2'], e.composition['H2O'], e.composition['CO2']])   
gas_transport = cool.TransportProperties(model = "thermo", thermo_object = thermo_gas, force_phase = 'g')
#gas_transport = cool.TransportProperties(model = "custom", custom_mu = 7.5e-5, custom_k =  0.20, custom_Pr = 0.62)
coolant_transport = cool.TransportProperties(model = "thermo", thermo_object = thermo_coolant)

#Add the cooling jacket
engine.add_cooling_jacket(wall_material, 
                               inlet_T, 
                               inlet_p0, 
                               coolant_transport, 
                               mdot_coolant, 
                               configuration = "vertical", 
                               channel_height = channel_height)

#Add the exhaust gas transport properties
engine.add_exhaust_transport(gas_transport)

'''Graphite insert'''
ablative_material = bam.materials.Graphite
engine.add_ablative(ablative_material)

#Print the wall and ablative material properties
print("Stainless steel properties:")
print(wall_material)

print("\n Graphite insert properties:")
print(ablative_material)
print("")

#Plot the geometry
engine.plot_geometry()
plt.show()

#Run simulation - use the Bartz with sigma for h_gas, and Sieder-Tate for h_coolant.
data = engine.steady_heating_analysis(to_json = False, h_gas_model = '3', h_coolant_model = '2')


In [ ]:
#Show plots
bam.plot.plot_temperatures(data, show_gas=False, show_ablative = False)
bam.plot.plot_temperatures(data, show_gas=True)
bam.plot.plot_jacket_pressure(data)
bam.plot.plot_resistances(data)
bam.plot.plot_qdot(data)
plt.show()

# Variable channel height, vertical channels

In [ ]:
import bamboo as bam
import bamboo.cooling as cool
import bamboo.materials

import numpy as np
import matplotlib.pyplot as plt
import pypropep as ppp
import bamboo.plot
import thermo
import time

'''Engine dimensions'''
Ac = np.pi*0.1**2               #Chamber cross-sectional area (m^2)
L_star = 1.5                    #L_star = Volume_c/Area_t
wall_thickness = 2e-3

'''Chamber conditions'''
pc = 15e5               #Chamber pressure (Pa)
p_tank = 50e5           #Tank / inlet coolant stagnation pressure (Pa) - used for cooling jacket
mdot = 5.4489           #Mass flow rate (kg/s)
p_amb = 1.01325e5       #Ambient pressure (Pa). 1.01325e5 is sea level atmospheric.
OF_ratio = 3.5          #Oxidiser/fuel mass ratio

'''We want to investigate adding water to the isopropyl alcohol'''
water_mass_fraction = 0.10  #Fraction of the fuel that is water, by mass

'''Coolant jacket'''
wall_material = bam.materials.CopperC700
mdot_coolant = mdot/(OF_ratio + 1) 
inlet_T = 298.15                    #Coolant inlet temperature

'''Get combustion properties from pypropep'''
#Initialise and get propellants
ppp.init()
e = ppp.Equilibrium()
ipa = ppp.PROPELLANTS['ISOPROPYL ALCOHOL']
water = ppp.PROPELLANTS['WATER']
n2o = ppp.PROPELLANTS['NITROUS OXIDE']

#Add propellants by mass fractions (note the mass fractions can add up to more than 1)
e.add_propellants_by_mass([(ipa, 1-water_mass_fraction), 
                        (water, water_mass_fraction), 
                        (n2o, OF_ratio)])

#Adiabatic combustion using chamber pressure                      
e.set_state(P = pc/1e5, type='HP')                      

gamma = e.properties.Isex   #I don't know why they use 'Isex' for gamma. 
cp = 1000*e.properties.Cp   #Cp is given in kJ/kg/K, we want J/kg/K
Tc = e.properties.T

'''Choose the models we want to use for transport properties of the coolant and exhaust gas'''
thermo_coolant = thermo.mixture.Mixture(['isopropanol', 'water'], ws = [1 - water_mass_fraction, water_mass_fraction])
thermo_gas = thermo.mixture.Mixture(['N2', 'H2O', 'CO2'], zs = [e.composition['N2'], e.composition['H2O'], e.composition['CO2']])   

gas_transport = cool.TransportProperties(model = "thermo", thermo_object = thermo_gas, force_phase = 'g')
coolant_transport = cool.TransportProperties(model = "thermo", thermo_object = thermo_coolant, force_phase = 'l')

'''Create the engine object'''
perfect_gas = bam.PerfectGas(gamma = gamma, cp = cp)    #Gas for frozen flow
chamber_conditions = bam.ChamberConditions(pc, Tc, mdot)
nozzle = bam.Nozzle.from_engine_components(perfect_gas, chamber_conditions, p_amb, type = "rao", length_fraction = 0.8)
white_dwarf = bam.Engine(perfect_gas, chamber_conditions, nozzle)
chamber_length = L_star*nozzle.At/Ac

'''Add the cooling system to the engine'''
white_dwarf.add_geometry(chamber_length, Ac, wall_thickness)
white_dwarf.add_exhaust_transport(gas_transport)
white_dwarf.add_ablative(bam.materials.Graphite, bam.materials.CopperC700, regression_rate = 0.0033e-3, xs = [-100, 100], ablative_thickness = None)

def get_data(channel_height):
    white_dwarf.add_cooling_jacket(wall_material, inlet_T, p_tank, coolant_transport, mdot_coolant, configuration = "vertical", channel_height = channel_height)

    '''Run the heating analysis'''
    #print(f"Sea level thrust = {white_dwarf.thrust(1e5)/1000} kN")
    #print(f"Sea level Isp = {white_dwarf.isp(1e5)} s")

    return white_dwarf.steady_heating_analysis(number_of_points = 250, to_json = False)

heights = np.linspace(0.5e-3, 5e-3, 20)
isps = np.zeros(len(heights))
max_inner_wall_Ts = np.zeros(len(heights))
pressure_drop = np.zeros(len(heights))

for i in range(len(heights)):
    data = get_data(heights[i])
    max_inner_wall_Ts[i] = np.amax(data["T_wall_inner"])
    pressure_drop[i] = data["p_coolant"][0] - data["p_coolant"][-1] 
    print(f"i = {i}, max T = {max_inner_wall_Ts[i]} K, pressure drop = {pressure_drop[i]/1e5} bar")


In [ ]:
fig, T_axs = plt.subplots()


p_axs = T_axs.twinx() 

T_axs.plot(heights/1e-3, max_inner_wall_Ts - 273.15, label = "Temperature (°C)", color = 'orange')
p_axs.plot(heights/1e-3, pressure_drop/1e5, label = "Pressure drop (bar)", color = 'green')

T_axs.grid()
T_axs.set_xlabel("Cooling channel height (mm)")
T_axs.set_ylabel("Maximum inner liner temperature (°C)")
p_axs.set_ylabel("Pressure drop (bar)")
T_axs.legend(loc='lower right')
p_axs.legend(loc='upper left')
plt.title("Vertical channels, copper walls, graphite insert")

fig.set_size_inches(12, 8)

plt.show()